In [ ]:
%pip install dash plotly

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from dash import Dash, dcc, html, Input, Output

In [ ]:
from environment import dh, pio_renderer
if pio_renderer is not None:
    pio.renderers.default = pio_renderer

In [ ]:
PROJECT_NAME = "AreaVerde"

In [ ]:
# Load data
project = dh.get_or_create_project(PROJECT_NAME)

spira = project.get_dataitem("spira_flow_data_2024").as_df()
accuracy = project.get_dataitem("spira_accur_data_2024").as_df()
spira['DateTime'] = pd.to_datetime(spira['DateTime'])
accuracy['DateTime'] = pd.to_datetime(accuracy['DateTime'])


In [ ]:
accuracy_pivot  = accuracy.copy()
accuracy_pivot = accuracy_pivot.pivot_table(index='DateTime', columns='spira_code', values='count')

In [ ]:
spira_codes = spira['spira_code'].unique().tolist()
accuracy_codes = accuracy_pivot.columns.unique().tolist()
extra_acc_codes = [code for code in accuracy_codes if code not in spira_codes]

In [ ]:
accuracy_filtered = accuracy[~accuracy['spira_code'].isin(extra_acc_codes)]

In [ ]:
# replacing first ip-address with the second used
accuracy_filtered.loc[accuracy_filtered['spira_code'] == '3.50 3.49 6 1', 'spira_code'] = '3.50 0.127 8 1' # 636
accuracy_filtered.loc[accuracy_filtered['spira_code'] == '4.2 4.48 2 1', 'spira_code'] = '4.48 4.48 2 1' # 733
accuracy_filtered.loc[accuracy_filtered['spira_code'] == '4.48 4.2 6 1', 'spira_code'] = '4.48 4.48 6 1' # 822

In [ ]:
spira.loc[spira['spira_unique_id']==733 ,'spira_code'] = '4.48 4.48 2 1'
spira.loc[spira['spira_unique_id']==822 ,'spira_code'] = '4.48 4.48 6 1'
spira.loc[spira['spira_unique_id']==636 ,'spira_code'] = '3.50 0.127 8 1'

In [ ]:
# Check for duplicates in spira
duplicates_in_spira = spira[spira.duplicated(subset=['spira_unique_id','spira_code', 'DateTime'], keep=False)]
print(f"Number of duplicates in spira: {len(duplicates_in_spira)}")

# Check for duplicates in acc_filtered
duplicates_in_acc = accuracy_filtered[accuracy_filtered.duplicated(subset=['spira_code', 'DateTime'], keep=False)]
print(f"Number of duplicates in acc_filtered: {len(duplicates_in_acc)}")


In [ ]:
duplicates_in_spira[duplicates_in_spira['spira_unique_id']==732] # This is an example that we can see duplicate rows

In [ ]:
spira_cleaned = spira.drop_duplicates(subset=['spira_unique_id','spira_code', 'DateTime'], keep='first')

In [ ]:
merged = spira_cleaned.merge(accuracy_filtered, on=['spira_code', 'DateTime'], suffixes=('_spira', '_acc'))

In [ ]:
off_acc = accuracy_filtered[accuracy_filtered['count']==-1]

In [ ]:
result = pd.merge(off_acc, spira_cleaned, on=['spira_code', 'DateTime'], how='inner')

In [ ]:
app = Dash(__name__)

app.layout = html.Div([
    html.H4('Interactive time-series plot with spira data'),
    dcc.Graph(id="time-series-plot"),
    html.P("Filter by DateTime:"),
    dcc.DatePickerRange(
        id='date-picker-range',
        start_date=result["DateTime"].min().date(),
        end_date=result["DateTime"].max().date(),
        display_format='YYYY-MM-DD',
        start_date_placeholder_text='YYYY-MM-DD',
    ),
])

@app.callback(
    Output("time-series-plot", "figure"),
    Input("date-picker-range", "start_date"),
    Input("date-picker-range", "end_date")
)
def update_time_series(start_date, end_date):
    if start_date is None or end_date is None:
        return px.line(title="Please select a date range.")

    mask = (result['DateTime'] >= start_date) & (result['DateTime'] <= end_date)
    filtered_result = result[mask]

    fig = px.scatter(
        filtered_result,
        x="DateTime",
        y="count_y",
        color="spira_code", 
        labels={
            "DateTime": "Datetime",
            "count_y": "Real Value",
            "spira_code": "spira_code"
        }
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
non_zero_acc = accuracy_filtered[accuracy_filtered['count']!=100]

In [ ]:
non_zero_result = pd.merge(non_zero_acc, spira_cleaned, on=['spira_code', 'DateTime'], how='inner',suffixes=('_acc', '_spira'))

In [ ]:
non_zero_result             

In [ ]:
app = Dash(__name__)

app.layout = html.Div([
    html.H4('Interactive time-series plot with spira data'),
    dcc.Graph(id="time-series-plot"),
    html.P("Filter by DateTime:"),
    dcc.DatePickerRange(
        id='date-picker-range',
        start_date=non_zero_result["DateTime"].min().date(),
        end_date=non_zero_result["DateTime"].max().date(),
        display_format='YYYY-MM-DD',
        start_date_placeholder_text='YYYY-MM-DD',
    ),
])

@app.callback(
    Output("time-series-plot", "figure"),
    Input("date-picker-range", "start_date"),
    Input("date-picker-range", "end_date")
)
def update_time_series(start_date, end_date):
    if start_date is None or end_date is None:
        return px.line(title="Please select a date range.")

    mask = (non_zero_result['DateTime'] >= start_date) & (non_zero_result['DateTime'] <= end_date)
    filtered_result = non_zero_result[mask]

    fig = px.scatter(
        filtered_result,
        x="DateTime",
        y="count_spira",
        color="spira_code",
        hover_data={"count_acc": True},
        labels={
            "DateTime": "Datetime",
            "count_spira": "Real Value",
            "spira_code": "spira_code"
        }
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)